In [1]:
import os
import json

import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import numpy as np

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [10]:
# for file in os.listdir('JSONs_for_TP'):
#     if file == 'earthquakes_original.json':
#         continue
        
#     converted_data = dict(data=list())
#     with open('JSONs_for_TP/' + file, 'r') as f:
#         json_data = json.loads(f.read())
#         keys = list(json_data.keys())    
#         converted_data['keys_'] = keys
        
#         for num in json_data[keys[0]].keys():
#             data = dict()
            
#             for key in keys:
#                 data[key] = json_data[key][num]
                
#             converted_data['data'].append(data)
            
#         f.close()
        
#     with open('d3_jsons/' + file, 'w') as write_file:
#         write_file.write(json.dumps(converted_data))
#         write_file.close()

In [3]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [4]:
%%javascript
// console.log(require.s.contexts._.defined);
require(['d3'], function(d3){
    console.log('THIS IS d3: ', d3);
    console.log('d3 version: ', d3.version);
   
});


<IPython.core.display.Javascript object>

In [5]:
%%html
<style>
// add your CSS styling here
h1 {
    font-family: sans-serif;
    font-weight: 300;
    font-size: 16px;
}
</style>

In [11]:
%%javascript
require(['d3'], function(d3) {
    
    var width = 800,
        height = 600;
    
    var hTagsFontFamily = 'sans-serif',
        hTagsFontWeight = '300';
   
    //a weird idempotency thing
    $('#chart1').remove();
    //create canvas
    element.append(`
        <div id='chart1' class='row'>
            <div id='map-title' class='row'><h1>Earthquakes 1900-2011</h1></div>            
            <div id='map-container' class='col-md-8'>
                <div id='map'></div>
            </div>
            <div id='stats-container' class='class='col-md-4'>
                <div class='row'>
                    <h2 id='stats-country'>Country: <span id='stats-country-dynamic'></span></h1>
                    <div id='stats-earthquake-dynamic' class='row'><h2 id='stats-earthquake'>Earthquake: </h2></div>
                </div>                
            </div>
        </div>
    `);
    $('#map-title h1').css({ 'font-family': hTagsFontFamily,  'font-weight': hTagsFontWeight, 'font-size': '40px'});  
    
    var projection = d3.geo.mercator()
        .scale(104)
        .translate([width / 2.45, height / 2]);
      
    var path = d3.geo.path().projection(projection);
  
    var svg = d3.select('#map').append('svg')
        .attr({
          width: width,
          height: height
        });

    d3.json('custom.geo.json', function(error, json) {
        console.log(json.features);
        
        // Draw each province as a path
        svg.selectAll('path')
            .data(json.features)
            .enter().append('path')
                .attr({
                    'd': path,
                    'id': function(d) {return d.properties.name;},
                    'class': function(d) { return ['continent', d.properties.continent]; }
                })
                .style({
                    'fill': '#666',
                    'stroke-opacity': .4,
                    'stroke': '#D3D3D3'
                })
                .on('mouseover', function(d) {
                    d3.select( this )
                        .transition()
                        .style({
                            'stroke-opacity': 1,
                            'stroke': '#f00',
                            'fill': 'pink'
                        });
            
                    $( '#stats-country-dynamic' ).text( d.properties.name ) ;
                })
                .on('mouseout', function(d) {
                    d3.select(this)
                        .transition()
                        .style({
                            'stroke-opacity': .4,
                            'stroke': '#D3D3D3',
                            'fill': '#666'
                        });
            
                    $('#stats-country-dynamic').text( ' ' );
                });
        
        d3.json('data/earthquakes_json.json', function(error, earthquakes_data) {            
            var converted_earthquakes = [];
            var eq_keys = Object.keys(earthquakes_data);
            
            for (var i = 0; i < Object.keys(earthquakes_data['lat']).length; i++) {
                var data_point = new Object();
                
                for (var j = 0; j < eq_keys.length; j++) {
                    var key = eq_keys[j];
                    var value = earthquakes_data[eq_keys[j]][i];
                    if ((key == 'Lat' || key == 'Long')) {
                        if ((value == '?' || value == '??' || value == 0)) {
                            data_point = 0;
                            break;
                        } else {
                            value = parseFloat(value);
                        }

                    }
                    
                    data_point[key] = value;
                }
                if (data_point !== 0) {
                    converted_earthquakes.push(data_point);
                }                
            }
            console.log('converted_earthquakes: ', converted_earthquakes);
            
            svg.selectAll('circle')
                .data(converted_earthquakes).enter()
                .append('circle')
                .attr({
                  cx: function(d) { return projection([d.Long, d.Lat])[0]; },
                  cy: function(d) { return projection([d.Long, d.Lat])[1]; },
                  r: function(d) { 
                      if (d.Magnitude == -1) {
                          return 0;
                      } else {
                          return Math.sqrt(d.Magnitude);;
                      }    
                  },
                  'class': 'earthquake'
                })
                .style('fill', 'red')
                .on('mouseover', function(d) {
                    console.log(d);
                
                    var $earthquakeStats = `
                        <ul>
                            <li><strong>Time of event: ${d.day}/${d.month}/${d.year}</strong></li>
                            <li><strong>Magnitude: ${d.Magnitude}</strong></li>
                            <li><strong>Total Deaths: ${d.deaths}</strong></li>
                            <li><strong>Depth: ${d['Depth (km)']}(km)</strong></li>
                        </ul>
                    `;
                
                    $( '#stats-earthquake-dynamic' ).append( $earthquakeStats );
                })
                .on('mouseout', function(d) {
                    $( '#stats-earthquake-dynamic ul' ).remove();
                });
            });
            
            console.log(svg.selectAll('circle'));
    });
});

<IPython.core.display.Javascript object>

In [32]:
%%javascript
require(['d3'], function(d3) {
    var data_pieces = { 
        earthquakes: { name: 'earthquakes', file_path: 'd3_jsons/earthquakes2.json', color: 'green' }, 
        tsunamis: { name: 'tsunamis', file_path: 'd3_jsons/tsunamis.json', color: 'blue' }, 
        eruptions: { name: 'eruptions', file_path: 'd3_jsons/eruptions.json', color: 'red' } 
    };
    
    $('#chart1').remove();
    element.append(`
        <div id='chart1' class='row'>
            <div id='map-title' class='row'><h1>Earthquakes 1900-2011</h1></div>            
            <div id='map-container' class='col-md-8'>
                <div id='map'></div>
            </div>
            <div id='stats-container' class='class='col-md-4'>
                <div class='row'>
                    <h2 id='stats-country'>Country: <span id='stats-country-dynamic'></span></h1>
                    <div id='stats-earthquake-dynamic' class='row'><h2 id='stats-earthquake'>Earthquake: </h2></div>
                </div>                
            </div>
        </div>
    `);

    var width = 800,
        height = 600;
    
    var projection = d3.geo.mercator()
        .scale(104)
        .translate([width / 2.45, height / 2]);
      
    var path = d3.geo.path().projection(projection);
  
    var svg = d3.select('#map').append('svg')
        .attr({
          width: width,
          height: height
        });

    d3.json('custom.geo.json', function(error, json) {
        console.log(json.features);
        
        // Draw each province as a path
        svg.selectAll('path')
            .data(json.features)
            .enter().append('path')
                .attr({
                    'd': path,
                    'id': function(d) {return d.properties.name;},
                    'class': function(d) { return ['continent', d.properties.continent]; }
                })
                .style({
                    'fill': '#666',
                    'stroke-opacity': .4,
                    'stroke': '#D3D3D3'
                })
                .on('mouseover', function(d) {
                    d3.select( this )
                        .transition()
                        .style({
                            'stroke-opacity': 1,
                            'stroke': '#f00',
                            'fill': 'pink'
                        });
            
                    $( '#stats-country-dynamic' ).text( d.properties.name ) ;
                })
                .on('mouseout', function(d) {
                    d3.select(this)
                        .transition()
                        .style({
                            'stroke-opacity': .4,
                            'stroke': '#D3D3D3',
                            'fill': '#666'
                        });
            
                    $('#stats-country-dynamic').text( ' ' );
                });

        for (const d in data_pieces) {
            var data = data_pieces[d];
            console.log('data', data);
            
            d3.json(data.file_path, function(error, json_data) {               
               console.log('json_data', json_data);

                svg.selectAll('circle')
                    .data(json_data.data).enter()
                    .append('circle')
                    .attr({
                      cx: function(d) { return projection([d.lng, d.lat])[0]; },
                      cy: function(d) { return projection([d.lng, d.lat])[1]; },
                      r: '2',
                      'class': 'circle-' + data.name
                    })
                    .style({
                        'fill': data.color,
                        'opacity': .6
                    })
                    .on('mouseover', function(d) {
                        console.log('Mouse over: ', d); 
                    })
                    .on('mouseout', function(d) {
                        console.log('Mouse out');
                    });
                });

                console.log(svg.selectAll('circle')); 
        };
    });
});

<IPython.core.display.Javascript object>